The dataset Tiny ImageNet has to be in the 'data/' folder

Instruction to download and extract the data:

wget http://cs231n.stanford.edu/tiny-imagenet-200.zip

unzip tiny-imagenet-200.zip

rm tiny-imagenet-200.zip

In [1]:
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

import os
import pandas as pd
import numpy as np
from PIL import Image

['/device:CPU:0', '/device:GPU:0']


# 0. Loading the data

### TinyImageNet does not have labels for the testing data. We will use the 10k validation images for the testing data.

In [2]:
#Generate dataset

###### TRAINING DATA #######
#Load Training images and labels
train_directory = "./data/tiny-imagenet-200/train" 
image_list=[]
label_list=[]

label_dic={} #convert label str to int from 0 



for l,sub_dir in enumerate(os.listdir(train_directory)):
    if not sub_dir in label_dic:
        label_dic[sub_dir]=l
    sub_dir_name=os.path.join(train_directory,sub_dir,'images')
    n=0
    for file in os.listdir(sub_dir_name):
        filename = os.fsdecode(file)
        if filename.endswith(".JPEG")  or filename.endswith(".jpg") or filename.endswith(".png"):
            img=np.array(Image.open(os.path.join(sub_dir_name,file)))
            if(img.shape == (64, 64, 3)): #removing images without 3 channels
                image_list.append(img)
                label_list.append(int(label_dic[sub_dir]))
                n+=1
    print(n,'pictures in folder',sub_dir,sep=' ' )
            
X_tot=np.array(image_list)
y_tot=np.array(label_list)

###### TEST DATA #######

#getting the labels from the txt file
df = pd.read_table('./data/tiny-imagenet-200/val/val_annotations.txt', header=None)
test_labels={} 
for index, row in df.iterrows():
    test_labels[str(row[0])]= row[1]


test_directory = "./data/tiny-imagenet-200/val/images" 

test_image_list=[]
test_label_list=[]

for file in os.listdir(test_directory):
    filename = os.fsdecode(file)
  
    if filename.endswith(".JPEG")  or filename.endswith(".jpg") or filename.endswith(".png"):
        img=np.array(Image.open(os.path.join(test_directory,file)))
        if(img.shape == (64, 64, 3)): #removing images without 3 channels
            test_image_list.append(img)
            test_label_list.append(label_dic[test_labels[filename]])
            
X_test=np.array(test_image_list)
y_test=tf.keras.utils.to_categorical(np.array(test_label_list))

500 pictures in folder n01443537
500 pictures in folder n01629819
500 pictures in folder n01641577
498 pictures in folder n01644900
496 pictures in folder n01698640
499 pictures in folder n01742172
492 pictures in folder n01768244
497 pictures in folder n01770393
497 pictures in folder n01774384
495 pictures in folder n01774750
500 pictures in folder n01784675
498 pictures in folder n01855672
499 pictures in folder n01882714


KeyboardInterrupt: 

In [4]:
shuffler=np.random.RandomState(seed=10).permutation(len(X_tot))
X_tot = X_tot[shuffler]
y_tot = y_tot[shuffler]
y_tot=tf.keras.utils.to_categorical(y_tot)

X_train = X_tot[5000:]
y_train =  y_tot[5000:]
X_val = X_tot[:5000]
y_val =  y_tot[:5000]


print('test data:')
print(X_test.shape)
print(y_test.shape)

print('train data:')
print(X_train.shape)
print(y_train.shape)

print('val data:')
print(X_val.shape)
print(y_val.shape)

NameError: name 'X_tot' is not defined

#### Saving the variables allows faster loading in the future

In [5]:
#save for future use without having to reload from the images folders
np.save('data/X_test.npy', X_test)
np.save('data/y_test.npy', y_test)
np.save('data/X_train.npy', X_train)
np.save('data/y_train.npy', y_train)
np.save('data/X_val.npy', X_val)
np.save('data/y_val.npy', y_val) 



NameError: name 'X_test' is not defined

In [2]:
X_test = np.load('data/X_test.npy')
y_test = np.load('data/y_test.npy')
X_train = np.load('data/X_train.npy')
y_train = np.load('data/y_train.npy')
X_val = np.load('data/X_val.npy')
y_val = np.load('data/y_val.npy')

# 1. Using ResNet18 with ImageNet

### We use our custom resnet model generator to get any resnet 

Imagenet has 200 different labels and is downsampled from ImageNet to a resolution of 64x64 

In [7]:
from utils.custom_resnet import custom_resnet18

resnet_18=custom_resnet18(input_shape=(64,64,3),n_classes=200)
resnet_18.summary()

Model: "custom_resnet18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
__________________________________________________________________________________________________
conv1_relu (ReLU)               (None, 32, 32, 64)   0           conv1_bn[0][0]                   
____________________________________________________________________________________

## 1.1 Resnet without data augmentation

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
base_learning_rate=0.6
epochs=45
batch_size=128

train_datagen = ImageDataGenerator(
    # no augmentation
        )

# CHange learning rate
def change_learning_rate(epoch, lr):
    if epoch % 14 == 0 and epoch:
        print('new learning rate: ',lr)
        return 0.1 *lr
    return lr

lr_callback = [LearningRateScheduler(change_learning_rate, verbose=1)]

#Early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=7)

resnet_18.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=base_learning_rate,momentum=0.9),loss='categorical_crossentropy', metrics = ['acc'])


train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
step_size_train=train_generator.n//train_generator.batch_size

history_resnet_18=resnet_18.fit(train_generator,
                   steps_per_epoch = step_size_train,
                   epochs = epochs,
                   validation_data=(X_val, y_val),
                   callbacks=[lr_callback,es])

Epoch 1/45

Epoch 00001: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 52s 61ms/step - loss: 5.7145 - acc: 0.0057 - val_loss: 5.2335 - val_acc: 0.0082
Epoch 2/45

Epoch 00002: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 40s 54ms/step - loss: 5.1289 - acc: 0.0118 - val_loss: 5.6034 - val_acc: 0.0120
Epoch 3/45

Epoch 00003: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 40s 55ms/step - loss: 4.8841 - acc: 0.0268 - val_loss: 5.0003 - val_acc: 0.0222
Epoch 4/45

Epoch 00004: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 41s 56ms/step - loss: 4.6534 - acc: 0.0446 - val_loss: 6.1477 - val_acc: 0.0242
Epoch 5/45

Epoch 00005: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 

#### How to save the model and reload it

In [9]:
# Evaluate the model
loss, acc = resnet_18.evaluate(X_test, y_test, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * acc))

# Save the weights

resnet_18.save_weights('./models/tinyImageNet/resnet_18')

# reload from saved weights:
model = custom_resnet18(input_shape=(64,64,3),n_classes=200)
model.load_weights('./models/tinyImageNet/resnet_18')

# Evaluate the model
model.compile(loss='categorical_crossentropy', metrics = ['acc'])
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

308/308 - 2s - loss: 6.1252 - acc: 0.2840
 model, accuracy: 28.40%
308/308 - 2s - loss: 6.1252 - acc: 0.2840
Restored model, accuracy: 28.40%


In [1]:
print(history_resnet_18.history.keys())
acc = history_resnet_18.history['acc']
val_acc = history_resnet_18.history['val_acc']

loss = history_resnet_18.history['loss']
val_loss = history_resnet_18.history['val_loss']

import matplotlib.pyplot as plt
plt.plot(acc, label='Training Accuracy', color="blue")
plt.plot(val_acc, label='Validation Accuracy', color="r")
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

NameError: name 'history_resnet_18' is not defined

Accuracy and loss graphs

In [ ]:
import matplotlib.pyplot as plt

acc = history_resnet_18.history['acc']
val_acc = history_resnet_18.history['val_acc']

loss = history_resnet_18.history['loss']
val_loss = history_resnet_18.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color="blue")
plt.plot(val_acc, label='Validation Accuracy', color="r")
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color="blue")
plt.plot(val_loss, label='Validation Loss', color="r")
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')

plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## 1.2 Resnet18 With data augmentation

In [8]:
from utils.custom_resnet import custom_resnet18
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

resnet_18_aug=custom_resnet18(input_shape=(64,64,3),n_classes=200)

In [9]:
base_learning_rate=0.6
epochs=45
batch_size=128

train_datagen = ImageDataGenerator(
    horizontal_flip=True,
    width_shift_range=8, #+-8 pixel shift max
    height_shift_range=8,#+-8 pixel shift max
    brightness_range=[0.8,1.2],
    rotation_range=2 #very little rotation
        )

# CHange learning rate
def change_learning_rate(epoch, lr):
    if epoch % 14 == 0 and epoch:
        print('new learning rate: ',lr)
        return 0.1 *lr
    return lr
lr_callback = [LearningRateScheduler(change_learning_rate, verbose=1)]
#Early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=7)

resnet_18_aug.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=base_learning_rate,momentum=0.9),loss='categorical_crossentropy', metrics = ['acc'])


train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
step_size_train=train_generator.n//train_generator.batch_size

history_resnet_18_aug=resnet_18_aug.fit(train_generator,
                   steps_per_epoch = step_size_train,
                   epochs = epochs,
                   validation_data=(X_val, y_val),
                   callbacks=[lr_callback,es])

Epoch 1/45

Epoch 00001: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 89s 113ms/step - loss: 5.8657 - acc: 0.0061 - val_loss: 5.1698 - val_acc: 0.0138
Epoch 2/45

Epoch 00002: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 76s 104ms/step - loss: 5.1088 - acc: 0.0140 - val_loss: 6.1985 - val_acc: 0.0094
Epoch 3/45

Epoch 00003: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 79s 108ms/step - loss: 4.8256 - acc: 0.0296 - val_loss: 4.6692 - val_acc: 0.0448
Epoch 4/45

Epoch 00004: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================] - 80s 109ms/step - loss: 4.5368 - acc: 0.0561 - val_loss: 4.5398 - val_acc: 0.0572
Epoch 5/45

Epoch 00005: LearningRateScheduler reducing learning rate to 0.6000000238418579.
727/727 [==============================

727/727 [==============================] - 79s 109ms/step - loss: 1.7168 - acc: 0.5593 - val_loss: 2.7091 - val_acc: 0.3998


In [10]:
# Evaluate the model
loss, acc = resnet_18_aug.evaluate(X_test, y_test, verbose=2)
print(" model, accuracy: {:5.2f}%".format(100 * acc))

# Save the weights

resnet_18_aug.save_weights('./models/tinyImageNet/resnet_18_aug')

# reload from saved weights:
model = custom_resnet18(input_shape=(64,64,3),n_classes=200)
model.load_weights('./models/tinyImageNet/resnet_18_aug')

# Evaluate the model
model.compile(loss='categorical_crossentropy', metrics = ['acc'])
loss, acc = model.evaluate(X_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

308/308 - 2s - loss: 2.7554 - acc: 0.3845
 model, accuracy: 38.45%
308/308 - 2s - loss: 2.7554 - acc: 0.3845
Restored model, accuracy: 38.45%


In [ ]:


import matplotlib
import numpy as np
import matplotlib.pyplot as plt
acc = [1,6,4,5,4,5,7,8]
val_acc = [1,1,2,5,4,5,7,8]


plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color="blue")
plt.plot(val_acc, label='Validation Accuracy', color="r")
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')
plt.show()

# 2. SE-ResNet18

## 2.1 SE-ResNet18 without data augmentation

In [3]:
from utils.SE_resnet import SE_resnet18

SE_resnet18=SE_resnet18(input_shape=(64,64,3),n_classes=200,name='SE_resnet18')
SE_resnet18.summary()

Model: "SE_ResNet18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[1][0]                 
__________________________________________________________________________________________________
conv1_relu (ReLU)               (None, 32, 32, 64)   0           conv1_bn[1][0]                   
________________________________________________________________________________________

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
base_learning_rate=0.6
epochs=45
batch_size=128

train_datagen = ImageDataGenerator(
    # no augmentation
        )

# CHange learning rate
def change_learning_rate(epoch, lr):
    if epoch % 14 == 0 and epoch:
        print('new learning rate: ',lr)
        return 0.1 *lr
    return lr

lr_callback = [LearningRateScheduler(change_learning_rate, verbose=1)]

#Early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=8)

SE_resnet18.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=base_learning_rate,momentum=0.9),loss='categorical_crossentropy', metrics = ['acc'])


train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
step_size_train=train_generator.n//train_generator.batch_size

history_SE_resnet18=SE_resnet18.fit(train_generator,
                   steps_per_epoch = step_size_train,
                   epochs = epochs,
                   validation_data=(X_val, y_val),
                   callbacks=[lr_callback,es])

Epoch 1/45

Epoch 00001: LearningRateScheduler reducing learning rate to 0.6000000238418579.
138/727 [====>.........................] - ETA: 35s - loss: 0.3671 - acc: 0.8883

KeyboardInterrupt: 